> Clasificación de imágenes

La clasificación de imágenes es una aplicación muy importante en aprendizaje automático: es lo que permite a los ordenadores "ver". Desde reconocer dígitos escritos a mano hasta identificar animales en fotos, el objetivo es enseñar a un modelo a asignar etiquetas a las imágenes basándose en lo que aprende de los ejemplos. En este *notebook*, construirás y probarás un clasificador de imágenes simple usando un conjunto de datos pequeño, explorando cómo las máquinas pueden aprender a distinguir entre dos tipos diferentes de objetos (¡¡por ejemplo, muffins y perros!!).

# Preparación del entorno

Todos los `import`s necesarios van aquí

In [ ]:
import pathlib
import zipfile

import fastai
from fastai.vision.all import *
from fastai.torch_core import set_seed

import torch

En principio, podrías ejecutar el *notebook* tanto en *Colab* como localmente. ¿Se está ejecutando el *notebook* en *Colab*?

In [ ]:
try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

running_in_colab

Si no estás ejecutando en *Colab*, puede que quieras elegir una GPU si hay varias disponibles. Ignora esto si estás ejecutando en *Colab*

In [ ]:
if not running_in_colab:

    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

¿Está disponible la aceleración por GPU?

In [ ]:
torch.cuda.is_available()

# Datos

Vamos a usar el dataset [Muffin vs chihuahua](https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification/data) de [Kaggle](https://www.kaggle.com/https://www.kaggle.com/). Descargamos el archivo *.zip* correspondiente y lo descomprimimos.

In [ ]:
# data will live inside this inside directory
data_dir = pathlib.Path('muffin_chihuahua')

# it it doesn't exist (from a previous run)...
if not data_dir.exists():

    # ...it is created
    data_dir.mkdir(exist_ok=True)

    # data is downloaded as a zip file, that will be named
    zip_file = data_dir / 'muffin-vs-chihuahua-image-classification.zip'

    # actual download
    !curl -L -o {zip_file} https://www.kaggle.com/api/v1/datasets/download/samuelcortinhas/muffin-vs-chihuahua-image-classification

    # data is unzipped inside the data directory
    with zipfile.ZipFile(zip_file, 'r') as zf:
        zf.extractall(data_dir)

¿Qué hay en el directorio?

In [ ]:
list(data_dir.iterdir())

Por lo tanto, además del *.zip* que acabamos de descargar, tenemos los conjuntos habituales *train* y *test*, y dentro de cada uno

In [ ]:
for e in ['train', 'test']:
    print(f"{e}:")
    for c in ['chihuahua', 'muffin']:
        n = len(list((data_dir / e / c).iterdir()))
        print(f"  {c}: {n}")    

Nos centraremos solo en los datos de entrenamiento

In [ ]:
data_root = data_dir / 'train'

## Carga de datos

Utilizaremos dos librerías de *deep learning* muy conocidas: [fastai](https://github.com/fastai/fastai) y [PyTorch](https://pytorch.org/).

Vamos a dejar que [fastai](https://github.com/fastai/fastai) lea las imágenes de las distintas carpetas (teniendo en cuenta las etiquetas) y cree automáticamente una división *train*/*validation*. Lo que hace el código de abajo, en resumen, es:

- Definir la(s) transformación(es) que se aplicarán a cada *item* individual (es decir, imagen). Redimensionaremos cada imagen a un tamaño manejable (por ejemplo, 256×256).

- Definir la(s) transformación(es) que se aplicarán a cada *batch*. Estas incluyen (importante!!) *augmentations* (consulta la [documentación](https://docs.fast.ai/vision.augment.html#aug_transforms) para ver qué es posible).

- Instanciar un objeto `DataBlock`, que es una especie de "plantilla" que especifica cómo acceder al dataset.

- Usar el `DataBlock` para obtener los `DataLoader`s (objetos de *PyTorch* pensados para recorrer el cojunto de datos batch a batch), uno para el conjunto de *train* y otro para el de *validation*.

Observa que se aplican dos operaciones de *resize*: una en `item_tfms` y otra en `batch_tfms`. No te preocupes por los detalles, pero la última es la que realmente determina el tamaño de las imágenes con las que se entrenará el modelo, y la primera es solo un paso de *predimensionado* que ayuda a evitar artefactos en las imágenes transformadas (no es necesario, pero se puede encontrar una explicación completa en el [libro de fastai](https://nbviewer.org/github/fastai/fastbook/blob/master/05_pet_breeds.ipynb#Presizing)).

In [ ]:
batch_size = 128

# a transformation to apply to every item
item_tfms = [Resize(256)]

# a `list` of transformations to apply to every *batch*
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),  # TODO: try a different seed
    item_tfms=item_tfms,
    batch_tfms=batch_tfms
)

dls = dblock.dataloaders(data_root, bs=batch_size)
dls.show_batch(max_n=12, figsize=(8,8))

Entre las transformaciones de *batch* (las pasadas mediante el parámetro `batch_tfms`) tenemos la función `aug_transforms`. Esta realiza [data augmentation](https://en.wikipedia.org/wiki/Data_augmentation). La idea es: una foto de un perro sigue siendo una foto de un perro si la rotas un poco...o la deformas un poco...o la recortas un poco (por ejemplo, te quedas solo con la cabeza). Entonces, cada una de estas *versiones* de la misma imagen se puede usar en el entrenamiento del modelo, y estamos generando (inventando) datos de entrenamiento de forma *artificial* (¡¡una cantidad infinita!!...ya que la cantidad de rotación/deformación/lo-que-sea se elegirá aleatoriamente en cada iteración del procedimiento de entrenamiento). *Data augmentation* es un truco muy común en visión por ordenador porque es muy útil (y computacionalmente barato).

<font color='red'>TO-DO</font>: Observa que hay una fuente adicional (además de la implícita en el uso de *data augmentation*) de *aleatoriedad* aquí: el parámetro `seed` pasado a `RandomSplitter`. ¿Para qué sirve?

# Entrenamiento del modelo

Vamos a hacer [transfer learning](https://en.wikipedia.org/wiki/Transfer_learning) explotando un modelo pre-entrenado como *mobilenet_v3_small*. Más sobre esto en cursos futuros...pero *transfer learning* nos permite reutilizar un modelo entrenado para una tarea en otra tarea diferente (pero relacionada). En este caso, estamos aprovechando un modelo entrenado para clasificar imágenes en [imagenet](https://www.image-net.org/)... que tiene 1.000 categorías.

La *métrica* que nos interesa es la precisión (*accuracy*).

In [ ]:
learn = vision_learner(dls, mobilenet_v3_small, metrics=accuracy)

Vamos a entrenar el modelo durante unas pocas *epochs*. Cada *epoch* recorre el dataset completo.

In [ ]:
learn.fine_tune(
    epochs=3,
    base_lr=3e-3
)

<font color='red'>TO-DO</font>: ¿Qué porcentaje de las veces se equivoca el modelo?

<font color='red'>TO-DO</font>: ¿Qué pasa si vuelves a entrenar el modelo (ejecutando otra vez el último par de celdas)? ¿Obtienes los mismos resultados? ¿Por qué no? (*Pista*: ¿cómo se inicializan los parámetros del modelo cada vez que llamas a `vision_learner`?)

# Resultados

Veamos algunas predicciones

In [ ]:
learn.show_results(max_n=9, figsize=(8,8))

Aunque el rendimiento es muy bueno, es interesante ver dónde el modelo tiene más dificultades.
Vamos a mirar la [matriz de confusión](https://es.wikipedia.org/wiki/Matriz_de_confusi%C3%B3n) para ver si al modelo se equivoca más con una clase que con otra.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(4,4), dpi=120)

<font color='red'>TO-DO</font>: A la vista de esta matriz de confusión, ¿el rendimiento del modelo depende de si la entrada es de una clase u otra?

Veamos algunas de las imágenes con las que el modelo tuvo más problemas.

In [ ]:
interp.plot_top_losses(4, nrows=2)

<font color='red'>TO-DO</font>: ¿Podrías adivinar la clase por tu cuenta?

# El modelo con tus propias imágenes

Elige una foto (muffin, chihuahua...u otra cosa) y pasásela al model para ver que predice. **La GUI requiere *Colab***.

In [ ]:
if running_in_colab:

    try:
        from google.colab import files
        up = files.upload()
        for fn in up:
            img = PILImage.create(fn)
            pred,pred_idx,probs = learn.predict(img)
            print(f'{fn} -> {pred}; probs={probs.tolist()}')
            display(img.to_thumb(256,256))
    except Exception as e:
        print('Local environment or no file uploaded:', e)

else:

    print("The GUI requires Colab.")

La probabilidad que obtienes es la de la primera etiqueta, es decir, la de un *chihuahua*.

<font color='red'>TO-DO</font>: ¿Qué pasa si le das al modelo algo que no es ni un muffin ni un chihuahua?

# Ajuste de hiperparámetros

## Augmentation

Vamos a experimentar con *data augmentation*...

<font color='red'>TO-DO</font>: Explora otros tipos de *augmentation* pasando diferentes parámetros a la función `aug_transforms`

In [ ]:
# aug = aug_transforms(...

Necesitamos crear un nuevo `DataBlock` (diferente), que se puede construir modificando el anterior. Se obtienen `DataLoaders` actualizados a partir de él.

In [ ]:
aug_dblock = dblock.new(item_tfms=item_tfms,batch_tfms=[*aug, Normalize.from_stats(*imagenet_stats)])
aug_dls = aug_dblock.dataloaders(data_root, bs=batch_size)

Vamos a visualizar un batch con las nuevas *augmentations* para ver es aspecto que tienen los datos "nuevos"...

In [ ]:
aug_dls.show_batch(max_n=12, figsize=(8,8))

...antes de entrenar

In [ ]:
aug_learn = vision_learner(aug_dls, mobilenet_v3_small, metrics=accuracy)
aug_learn.fine_tune(epochs=2, base_lr=3e-3)  # quick test
print('Accuracy:', aug_learn.validate()[1])

<font color='red'>TO-DO</font>: ¿Obtienes mejores resultados?

## Arquitectura diferente

<font color='red'>TO-DO</font>: Prueba un modelo *más grande*, como uno de la familia *resnet* (por ejemplo, `resnet50`). Puedes quedarte con el `DataBlock` original. ¿Vale la pena, considerando el aumento en el tiempo de entrenamiento?

Debería ser posible obtener una lista incompleta de modelos disponibles usando
```
import timm
timm.list_models()
```

# Sample questions

## What is the main goal of the image classification model described here?
- [ ] To make image files smaller so they fit on disk
- [ ] To decide which of two labels best matches a picture (for example, which kind of object it shows)
- [ ] To turn all color pictures into black-and-white
- [ ] To draw new pictures from scratch

## Why are the images split into a training set and a validation set?
- [ ] So the model can be trained on one part and then checked on images it has not seen
- [ ] So that each image gets two different labels
- [ ] So that half of the images can be safely deleted
- [ ] So that the images can be sorted by file name